In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

DEMO_LENGTH = 10

In [2]:
#test script for grabbing webcam frames

cap = cv2.VideoCapture(0)

for i in range(0, 100):
	res, frame = cap.read()
	if res:
		cv2.imshow("frame", frame)
		cv2.waitKey(1)
	else:
		print("Couldn't grab frame\n")

cv2.destroyAllWindows()
cap.release()

In [ ]:
model_name = 'hand_landmark.tflite'
hand_model = tf.lite(model_name)

In [ ]:
# Setup Google Hands API and initialize the video stream

# initialize video stream 
cap = cv2.VideoCaputre(0) #Note, you need a webcam to be plugged in

# setup google hands api
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.75, min_tracking_confidence=0.75)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
# Main running loop

for i in range(0, DEMO_LENGTH):
	res, frame = cap.read()
	if not res:
		print("Coulnd't grab frame, continuing to next iteration")
	else:
		# Flip across y axis, required as per documentation
		frame = cv2.flip(frame, 1)
		# Convert to RGB (opencv is natively BGR)
		frame = cv2.cvtColor(frame, cv2.BGR2RGB)
		# Make NN call 
		results = hands.process(frame)
		# Perform Inference
